In [277]:
# -*- coding: utf8 -*-
#########################IMPORTS############################################
from os.path import expanduser
import codecs, optparse
import re
import random as rd
import sys,os,time
import string
import yaml, YamlDuplicates
import ParFuMor as PFM
from ParFuMor import *
import pickle

In [278]:
from string import translate, maketrans

In [279]:
home = expanduser("~")
annee="19"
def getSerie(nomKalaba):
    return home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/%s-%s/"%(annee,nomKalaba)

prevSerie=getSerie("K2")
nextSerie=getSerie("K1")

nomsPropres=["Kaleb","Mahira","Violette","Nabil","Katisha","Nicole"]
nomsPropresP=["kaleb","maira","vjolet","nabil","katiSa","nikol"]


In [280]:
with open(prevSerie+"Stems.yaml", 'r') as stream:
    prevStems=yaml.load(stream)
with open(nextSerie+"Stems.yaml", 'r') as stream:
    nextStems=yaml.load(stream)

In [281]:
def getLeaves(stemStructure):
    formes={}
    for element in stemStructure:
        if type(stemStructure[element])==list:
            formesCle=", ".join(stemStructure[element])
            formes[formesCle]=element
        else:
            formes.update(getLeaves(stemStructure[element]))
    return formes

In [282]:
prevLexique=getLeaves(prevStems)
nextLexique=getLeaves(nextStems)

In [283]:
def replace_keys(old_dict, key_dict):
    new_dict = { }
    for key in old_dict.keys():
        new_key = key_dict.get(key, key)
        if isinstance(old_dict[key], dict):
            new_dict[new_key] = replace_keys(old_dict[key], key_dict)
        else:
            new_dict[new_key] = old_dict[key]
    return new_dict


In [284]:
with open(prevSerie+"Phonology.yaml", 'r') as stream:
    phonology=yaml.load(stream)

In [285]:
obstrusivesSourdes=u"ptkfTsS"
obstrusivesVoisees=u"bdgvDzZ"

voisement=maketrans(obstrusivesSourdes,obstrusivesVoisees)
devoisement=maketrans(obstrusivesVoisees,obstrusivesSourdes)


consonnes=phonology["consonnes"]
voyelles=phonology["voyelles"]

In [286]:
def devoiserFinale(radical):
    if radical[-1] in obstrusivesVoisees:
        return radical.translate(devoisement)
    else:
        return None

def voiserVCV(radical):
    result=list(radical)
    cibles=[]
    for p,l in enumerate(radical[1:-1]):
        if l in obstrusivesSourdes and radical[p] in voyelles and radical[p+2] in voyelles:
            cibles.append(p+1)
    if cibles:
        cible=rd.choice(cibles)
        result[cible]=result[cible].translate(voisement)
        return "".join(result)
    else:
        return None

def metathese(radical):
    result=list(radical)
    cibles=[]
    for p,l in enumerate(radical):
        if l in consonnes:
            cibles.append(p)
    if len(cibles)>1 and rd.randint(1,3)==1:
        cible1=rd.choice(cibles[:-1])
        cible2=cibles[cibles.index(cible1)+1]
        result[cible1]=radical[cible2]
        result[cible2]=radical[cible1]
        result="".join(result)
        if result!=radical:
            return result
        else:
            return None
    else:
        return None

    
def apophonie(radical,choice="rdVowel"):
    result=list(radical)
    cibles=[]
    if not choice:
        choice=rd.choice(["apophonie","rdVowel"])
    if choice=="apophonie":
        for p,l in enumerate(radical):
            if l in "iae":
                cibles.append(p)
        if cibles:
            cible=rd.choice(cibles)
            result[cible]=phonology["apophonies"][result[cible]]
            return "".join(result)
        else:
            return None
    else:
        for p,l in enumerate(radical):
            if l in voyelles:
                cibles.append(p)
        if cibles:
            cible=rd.choice(cibles)
            result[cible]=rd.choice(voyelles)
            return "".join(result)
        else:
            return None

def changerLongueur(radical):
    result=list(radical)
    if len(radical)==5:
        if radical[0] in consonnes:
            if radical[-1] in consonnes:
                if rd.randint(1,5)==1:
                    result.append(rd.choice(voyelles))
                    return "".join(result)
                else:
                    return None
            else:
                if rd.randint(1,5)==1:
                    result.append(rd.choice(consonnes))
                    return "".join(result)
                else:
                    return None
        else:
            if rd.randint(1,3)==1:
                result.insert(0,rd.choice(consonnes))
                return "".join(result)
            else:
                return None
    elif len(radical)>5:
        if rd.randint(1,3)==1:
            if radical[0] in consonnes:
                result=result[:-1]
            else:
                result=result[1:]
            return "".join(result)
        else:
            return None
    else:
        if rd.randint(1,3)==1:
            if radical[0] in voyelles:
                result.insert(0,rd.choice(consonnes))
            elif radical[-1] in voyelles:
                result.append(rd.choice(consonnes))
            else:
                result.append(rd.choice(voyelles))
            return "".join(result)
        else:
            return None
        
            

In [287]:
apophonie("kulubu",choice="rnd")

'kulobu'

In [288]:
operations=[
    devoiserFinale,
    apophonie,apophonie,
    voiserVCV,voiserVCV,
    metathese,
    changerLongueur
           ]

In [289]:
def modifierRadical(radical):
    nOperations=rd.choice([0,1,1,1,2,2,2,3,3,3,4,5])
    while nOperations>0:
        operation=rd.choice(operations)
        result=operation(radical)
        if result:
            radical=result
            nOperations-=1
    return radical

In [290]:
racineChanges={}
for mot in prevLexique:
    if mot in nextLexique:
        if prevLexique[mot] in nomsPropresP:
            racineChanges[nextLexique[mot]]=prevLexique[mot]
        else:
            racineChanges[nextLexique[mot]]=modifierRadical(prevLexique[mot])
        print prevLexique[mot],racineChanges[nextLexique[mot]]
    else:
        print "***"
        print mot,"n'est pas dans le nouveau lexique"
        print "***"

for mot in nextLexique:
    if not mot in prevLexique:
        print "***"
        print mot,u"n'était pas dans le lexique précédent"
        print "***"

tujen tejan
nogida nadiga
maZura meZura
***
villageois.HYPER, villagEOIs, villagEOIS n'est pas dans le nouveau lexique
***
leNuNu leNoNu
wonije wonije
siran saran
lomot lamot
toNoju toNojo
neTim naDom
rapap rabap
kaleb kaleb
lijuli lelij
ZagoN ZageN
***
loup, loups n'est pas dans le nouveau lexique
***
Zodet Zadet
rusoj zurij
woreZa woreZo
veniS vunuS
neboru naboru
wujeT wajeT
burin ruboni
turona redona
bute bude
negele nugele
mogel mogel
raDik roDek
rageka ragega
mobuk mubok
nonom nonom
wuDora wuDor
wubuNu wubuNu
geZoj geZoj
Seduk Seduk
rulude rulude
Tojen Tajen
SoTew SuDew
roben roben
Zabar Zarab
tudep todepo
la la
temune temuni
najok najok
suzen sazen
legor legar
vjolet vjolet
ruS reS
***
villageois, villageoiS n'est pas dans le nouveau lexique
***
kodaw kodow
***
loup.HYPER, lOUP, lOUPs n'est pas dans le nouveau lexique
***
Sogup Sopig
kuziDa kuzeDi
moriz romes
TuNum TiNum
Sol Sol
godare gorada
zamiN zaNim
be bu
ziver ziver
dolut dolit
ZukuNo ZeNuko
digare digare
jeleno jeleni
kifi

In [291]:
for i in range(10):
    print modifierRadical("Tonif")

Tonafe
Tonuf
Tafan
Tinif
Tonif
Tonaf
nuDof
Tenaf
noDivu
Tinif


In [292]:
racineChanges

{'DugiN': 'gamaN',
 'NaZudo': 'ruzam',
 'Namuru': 'ladaj',
 'Nazuva': 'legar',
 'Netaf': 'lomase',
 'Nizunu': 'megif',
 'Nuben': 'zugama',
 'Nudazo': 'Todike',
 'Nuwu': 'reS',
 'Sazefe': 'goguto',
 'Sotuf': 'meZura',
 'Suraf': 'Nabul',
 'TaZuk': 'redona',
 'TewuN': 'zaNim',
 'Tiduti': 'dolit',
 'Tud': 'soDir',
 'Tudit': 'woZil',
 'Zabur': 'deDor',
 'ZavuN': 'mogel',
 'ZogeZu': 'kodow',
 'Zudogi': 'lumor',
 'b': 'bu',
 'bagad': 'sazen',
 'besuk': 'jumaS',
 'biNod': 'ridiNi',
 'biZuS': 'jasom',
 'bofegu': 'naloma',
 'bopev': 'roben',
 'bowego': 'mubok',
 'buDiw': 'faSaz',
 'bugew': 'ZageN',
 'bumad': 'zurij',
 'demuno': 'jiron',
 'dizuD': 'kuZaN',
 'dozis': 'jeleni',
 'dunage': 'TiNum',
 'duwoja': 'lamot',
 'feluS': 'lenad',
 'fesuS': 'nadiga',
 'fiDup': 'tejan',
 'fobago': 'munum',
 'g': 'la',
 'gaboz': 'rijan',
 'gafoZe': 'jagi',
 'gekid': 'nopuD',
 'gofazu': 'leNoNu',
 'gudeN': 'lireNa',
 'gukawu': 'nedar',
 'jowudo': 'Tudi',
 'juZezu': 'muzul',
 'judof': 'woreZo',
 'judur': 'raDil',


In [293]:
newStems=replace_keys(nextStems,racineChanges)

In [294]:
with open(nextSerie+'flowStems.yaml', 'w') as output:
    yaml.dump(newStems, output, default_flow_style=False,allow_unicode=True)

In [295]:
with open(nextSerie+"flowStems.yaml", 'r') as input:
    yamlLines=input.readlines()

In [296]:
yamlText="".join(yamlLines)
yamlText=re.sub(r"\n\s*-\s*",", ",yamlText)
yamlText=re.sub(r":,\s*",": ",yamlText)
yamlText=re.sub(r": *([^\n]+)",": [\g<1>]",yamlText)
print yamlText

ADJ:
  Tajen: [suivant, suivants, suivante, suivantes]
  TiNum: [courageux, courageuse, courageuses]
  digare: [terrible, terribles]
  jarami: [quatre]
  jumaS: [grand, grande, grands, grandes]
  korat: [blanc, blanche, blancs, blanches]
  linoj: [petit, petite, petits, petites]
  menuD: [rouge, rouges]
  nedar: [profond, profonds, profonde, profondes]
  pudir: [noir, noire, noirs, noires]
  ragega: [gros, grosse, grosses]
  redona: [bas, basse, basses]
  ridiNi: [trois]
  roben: [blessé, blessés, blessée, blessées]
  ruboni: [différent, différents, différente, différentes]
  sirus: [furieux, furieuse, furieuses]
  woZil: [maigre, maigres]
  zaNim: [jaune, jaunes]
  ziver: [effrayé, effrayés, effrayée, effrayées]
DET:
  bu: [DEM, ce, cet, cette, ces]
  kaV: [IND, un, une, des, dE]
  la: [DEF, le, la, les, l]
NOM:
  F:
    Nabul: [fruit, fruits]
    ZageN: [fille, filles]
    biliw: [chatte, chattes]
    burun: [croc, crocs]
    diwiN: [table, tables]
    faSaz: [forêt, forêts]
    geZo

In [297]:
with open(nextSerie+'newStems.yaml', 'w') as output:
    output.write(yamlText)